In [7]:

import os
import sys
import subprocess
from git import Repo, InvalidGitRepositoryError
from pathlib import Path
import time
import json
import requests
from datetime import datetime

In [8]:
#!/usr/bin/env python3
"""
Complete Git Operations Tool
This script provides a comprehensive set of Git operations including:
- Auto commit and push files
- Branch management
- Pull requests (GitHub integration)
- Merge operations
- Git log viewing
- And many more Git features
"""


class GitOperationsTool:
    def __init__(self):
        self.repo = None
        self.repo_url = None
        
    def get_repo_url(self):
        """Get repository URL from user input"""
        while True:
            repo_url = input("Enter the Git repository URL (SSH or HTTPS): ").strip()
            if repo_url:
                self.repo_url = repo_url
                return repo_url
            print("Please enter a valid repository URL.")

    def initialize_or_clone_repo(self, repo_url, local_path="."):
        """Initialize or clone the repository"""
        try:
            # Try to open existing repo
            self.repo = Repo(local_path)
            print(f"✓ Found existing Git repository at {local_path}")
            
            # Add remote if it doesn't exist
            try:
                origin = self.repo.remote('origin')
                if origin.url != repo_url:
                    print(f"Warning: Remote origin URL differs from provided URL")
                    print(f"Existing: {origin.url}")
                    print(f"Provided: {repo_url}")
            except:
                self.repo.create_remote('origin', repo_url)
                print(f"✓ Added remote origin: {repo_url}")
                
            return self.repo
            
        except InvalidGitRepositoryError:
            # Initialize new repo
            print(f"Initializing new Git repository at {local_path}")
            self.repo = Repo.init(local_path)
            self.repo.create_remote('origin', repo_url)
            print(f"✓ Created new repository with remote: {repo_url}")
            return self.repo

    def get_all_files_and_folders(self, path="."):
        """Get all files and folders in the current directory"""
        items = []
        
        # Walk through directory tree
        for root, dirs, files in os.walk(path):
            # Skip .git directory
            if '.git' in root:
                continue
                
            # Add files
            for file in files:
                file_path = os.path.join(root, file)
                relative_path = os.path.relpath(file_path, path)
                # Skip hidden files and common unwanted files
                if not relative_path.startswith('.') or relative_path == '.gitignore':
                    items.append(('file', relative_path))
        
        return items

    def commit_and_push_item(self, item_type, item_path, branch='main'):
        """Commit and push a single item"""
        try:
            # Check if item still exists
            full_path = Path(self.repo.working_dir) / item_path
            if not full_path.exists():
                print(f"⚠ Skipping {item_path} - file/folder no longer exists")
                return False
            
            # Add the item to staging
            self.repo.index.add([item_path])
            
            # Check if there are any staged changes
            staged_files = self.repo.index.diff("HEAD") if self.repo.head.is_valid() else self.repo.index.diff(None)
            if not staged_files:
                print(f"⚠ No changes to commit for {item_path}")
                return False
            
            # Create commit message
            commit_msg = f"Add {item_type}: {item_path}"
            
            # Commit
            commit = self.repo.index.commit(commit_msg)
            print(f"✓ Committed {item_type}: {item_path} (commit: {commit.hexsha[:8]})")
            
            # Push to remote
            origin = self.repo.remote('origin')
            
            # Ensure we're on the correct branch
            try:
                current_branch = self.repo.active_branch.name
                if current_branch != branch:
                    try:
                        self.repo.git.checkout(branch)
                    except:
                        # Create branch if it doesn't exist
                        self.repo.git.checkout('-b', branch)
            except:
                # For new repos, create the main branch
                self.repo.git.checkout('-b', branch)
            
            # Push to remote
            origin.push(refspec=f'{branch}:{branch}')
            print(f"✓ Pushed {item_type}: {item_path} to remote/{branch}")
            
            return True
            
        except Exception as e:
            print(f"✗ Error processing {item_path}: {str(e)}")
            return False

    def create_branch(self, branch_name):
        """Create a new branch"""
        try:
            # Create new branch
            new_branch = self.repo.create_head(branch_name)
            new_branch.checkout()
            print(f"✓ Created and switched to branch: {branch_name}")
            return True
        except Exception as e:
            print(f"✗ Error creating branch {branch_name}: {str(e)}")
            return False

    def list_branches(self):
        """List all branches"""
        try:
            print("\nLocal Branches:")
            for branch in self.repo.branches:
                current = "* " if branch == self.repo.active_branch else "  "
                print(f"{current}{branch.name}")
            
            print("\nRemote Branches:")
            for ref in self.repo.remote().refs:
                print(f"  {ref.name}")
            return True
        except Exception as e:
            print(f"✗ Error listing branches: {str(e)}")
            return False

    def checkout_branch(self, branch_name):
        """Checkout to a specific branch"""
        try:
            # Check if branch exists locally
            if branch_name in [b.name for b in self.repo.branches]:
                self.repo.git.checkout(branch_name)
                print(f"✓ Switched to existing branch: {branch_name}")
            else:
                # Try to checkout from remote
                try:
                    self.repo.git.checkout('-b', branch_name, f'origin/{branch_name}')
                    print(f"✓ Created and switched to branch: {branch_name} (from remote)")
                except:
                    # Create new branch
                    self.repo.git.checkout('-b', branch_name)
                    print(f"✓ Created and switched to new branch: {branch_name}")
            return True
        except Exception as e:
            print(f"✗ Error checking out branch {branch_name}: {str(e)}")
            return False

    def delete_branch(self, branch_name):
        """Delete a branch"""
        try:
            if branch_name == self.repo.active_branch.name:
                print(f"✗ Cannot delete current branch {branch_name}")
                return False
            
            self.repo.delete_head(branch_name, force=True)
            print(f"Deleted branch: {branch_name}")
            return True
        except Exception as e:
            print(f"✗ Error deleting branch {branch_name}: {str(e)}")
            return False

    def merge_branch(self, branch_name):
        """Merge a branch into current branch"""
        try:
            current_branch = self.repo.active_branch.name
            merge_branch = self.repo.heads[branch_name]
            
            # Perform merge
            self.repo.git.merge(branch_name)
            print(f"Merged {branch_name} into {current_branch}")
            return True
        except Exception as e:
            print(f"✗ Error merging branch {branch_name}: {str(e)}")
            return False

    def pull_changes(self, branch='main'):
        """Pull changes from remote repository"""
        try:
            origin = self.repo.remote('origin')
            origin.pull(branch)
            print(f"Pulled changes from remote/{branch}")
            return True
        except Exception as e:
            print(f"✗ Error pulling changes: {str(e)}")
            return False

    def push_changes(self, branch=None):
        """Push changes to remote repository"""
        try:
            if branch is None:
                branch = self.repo.active_branch.name
            
            origin = self.repo.remote('origin')
            origin.push(refspec=f'{branch}:{branch}')
            print(f"Pushed changes to remote/{branch}")
            return True
        except Exception as e:
            print(f"✗ Error pushing changes: {str(e)}")
            return False

    def show_status(self):
        """Show repository status"""
        try:
            print(f"\nRepository Status:")
            print(f"Current branch: {self.repo.active_branch.name}")
            
            # Show modified files
            modified_files = [item.a_path for item in self.repo.index.diff(None)]
            if modified_files:
                print(f"Modified files: {', '.join(modified_files)}")
            
            # Show staged files
            staged_files = [item.a_path for item in self.repo.index.diff("HEAD")]
            if staged_files:
                print(f"Staged files: {', '.join(staged_files)}")
            
            # Show untracked files
            untracked_files = self.repo.untracked_files
            if untracked_files:
                print(f"Untracked files: {', '.join(untracked_files)}")
            
            if not modified_files and not staged_files and not untracked_files:
                print("Working directory clean")
            
            return True
        except Exception as e:
            print(f"✗ Error showing status: {str(e)}")
            return False

    def show_log(self, limit=10):
        """Show commit log"""
        try:
            print(f"\nCommit Log (last {limit} commits):")
            commits = list(self.repo.iter_commits(max_count=limit))
            
            for commit in commits:
                print(f"Commit: {commit.hexsha[:8]}")
                print(f"Author: {commit.author.name} <{commit.author.email}>")
                print(f"Date: {datetime.fromtimestamp(commit.committed_date)}")
                print(f"Message: {commit.message.strip()}")
                print("-" * 50)
            
            return True
        except Exception as e:
            print(f"✗ Error showing log: {str(e)}")
            return False

    def create_pull_request(self, title, body, head_branch, base_branch='main'):
        """Create a pull request (GitHub only)"""
        try:
            # Extract GitHub info from repo URL
            if 'github.com' not in self.repo_url:
                print("✗ Pull requests are only supported for GitHub repositories")
                return False
            
            # Parse GitHub URL
            repo_path = self.repo_url.replace('https://github.com/', '').replace('.git', '')
            owner, repo_name = repo_path.split('/')
            
            # GitHub API token (user should set this)
            token = input("Enter your GitHub Personal Access Token: ").strip()
            if not token:
                print("✗ GitHub token is required for pull requests")
                return False
            
            # Create PR via GitHub API
            api_url = f"https://api.github.com/repos/{owner}/{repo_name}/pulls"
            headers = {
                'Authorization': f'token {token}',
                'Accept': 'application/vnd.github.v3+json'
            }
            
            data = {
                'title': title,
                'body': body,
                'head': head_branch,
                'base': base_branch
            }
            
            response = requests.post(api_url, headers=headers, json=data)
            
            if response.status_code == 201:
                pr_data = response.json()
                print(f"✓ Pull request created: {pr_data['html_url']}")
                return True
            else:
                print(f"✗ Error creating pull request: {response.json()}")
                return False
                
        except Exception as e:
            print(f"✗ Error creating pull request: {str(e)}")
            return False

    def stash_changes(self):
        """Stash current changes"""
        try:
            self.repo.git.stash('push', '-m', f'Stash created at {datetime.now()}')
            print("Changes stashed")
            return True
        except Exception as e:
            print(f"✗ Error stashing changes: {str(e)}")
            return False

    def apply_stash(self):
        """Apply latest stash"""
        try:
            self.repo.git.stash('pop')
            print("Stash applied")
            return True
        except Exception as e:
            print(f"✗ Error applying stash: {str(e)}")
            return False

    def auto_commit_and_push(self):
        """Auto commit and push all files"""
        print("\nDiscovering files and folders...")
        items = self.get_all_files_and_folders()
        
        if not items:
            print("No files or folders found to commit.")
            return
        
        print(f"Found {len(items)} items to process:")
        for item_type, item_path in items:
            print(f"  - {item_type}: {item_path}")
        
        # Confirm before proceeding
        print(f"\nThis will create {len(items)} separate commits and push them one by one.")
        confirm = input("Do you want to proceed? (y/N): ").strip().lower()
        
        if confirm != 'y':
            print("Operation cancelled.")
            return
        
        # Process each item
        print(f"\nProcessing {len(items)} items...")
        success_count = 0
        
        for i, (item_type, item_path) in enumerate(items, 1):
            print(f"\n[{i}/{len(items)}] Processing {item_type}: {item_path}")
            
            if self.commit_and_push_item(item_type, item_path):
                success_count += 1
            
            # Small delay between operations
            if i < len(items):
                time.sleep(0.5)
        
        # Summary
        print(f"\nSummary:")
        print(f"  - Successfully processed: {success_count}/{len(items)} items")
        print(f"  - Failed: {len(items) - success_count}/{len(items)} items")

    def show_menu(self):
        """Show main menu"""
        print("\nGit Operations Tool")
        print("=" * 50)
        print("1.  Auto commit and push all files")
        print("2.  Create branch")
        print("3.  List branches")
        print("4.  Checkout branch")
        print("5.  Delete branch")
        print("6.  Merge branch")
        print("7.  Pull changes")
        print("8.  Push changes")
        print("9.  Show status")
        print("10. Show commit log")
        print("11. Create pull request")
        print("12. Stash changes")
        print("13. Apply stash")
        print("14. Exit")
        print("=" * 50)

    def run(self):
        """Main application loop"""
        print("Git Operations Tool")
        print("=" * 50)
        
        # Get repository URL and initialize
        repo_url = self.get_repo_url()
        
        try:
            self.initialize_or_clone_repo(repo_url)
        except Exception as e:
            print(f"✗ Error initializing repository: {str(e)}")
            sys.exit(1)
        
        # Main menu loop
        while True:
            self.show_menu()
            
            try:
                choice = input("\nEnter your choice (1-14): ").strip()
                
                if choice == '1':
                    self.auto_commit_and_push()
                elif choice == '2':
                    branch_name = input("Enter branch name: ").strip()
                    self.create_branch(branch_name)
                elif choice == '3':
                    self.list_branches()
                elif choice == '4':
                    branch_name = input("Enter branch name to checkout: ").strip()
                    self.checkout_branch(branch_name)
                elif choice == '5':
                    branch_name = input("Enter branch name to delete: ").strip()
                    self.delete_branch(branch_name)
                elif choice == '6':
                    branch_name = input("Enter branch name to merge: ").strip()
                    self.merge_branch(branch_name)
                elif choice == '7':
                    branch = input("Enter branch name (default: main): ").strip() or 'main'
                    self.pull_changes(branch)
                elif choice == '8':
                    branch = input("Enter branch name (current branch): ").strip() or None
                    self.push_changes(branch)
                elif choice == '9':
                    self.show_status()
                elif choice == '10':
                    limit = input("Enter number of commits to show (default: 10): ").strip()
                    limit = int(limit) if limit.isdigit() else 10
                    self.show_log(limit)
                elif choice == '11':
                    title = input("Enter PR title: ").strip()
                    body = input("Enter PR description: ").strip()
                    head_branch = input("Enter head branch: ").strip()
                    base_branch = input("Enter base branch (default: main): ").strip() or 'main'
                    self.create_pull_request(title, body, head_branch, base_branch)
                elif choice == '12':
                    self.stash_changes()
                elif choice == '13':
                    self.apply_stash()
                elif choice == '14':
                    print("Goodbye!")
                    break
                else:
                    print("Invalid choice. Please try again.")
                    
            except KeyboardInterrupt:
                print("\n\nOperation cancelled by user.")
                break
            except Exception as e:
                print(f"✗ Error: {str(e)}")

In [9]:
if __name__ == "__main__":
    tool = GitOperationsTool()
    tool.run()

Git Operations Tool
✓ Found existing Git repository at .

Git Operations Tool
1.  Auto commit and push all files
2.  Create branch
3.  List branches
4.  Checkout branch
5.  Delete branch
6.  Merge branch
7.  Pull changes
8.  Push changes
9.  Show status
10. Show commit log
11. Create pull request
12. Stash changes
13. Apply stash
14. Exit

Discovering files and folders...
Found 40 items to process:
  - file: .gitignore
  - file: git_automate.ipynb
  - file: LICENSE
  - file: MANIFEST.in
  - file: pyproject.toml
  - file: README.md
  - file: requirements.txt
  - file: setup.py
  - file: dist\git_operations_tool-0.1.0-py3-none-any.whl
  - file: dist\git_operations_tool-0.1.0.tar.gz
  - file: git_operations_tool\main.py
  - file: git_operations_tool\__init__.py
  - file: git_operations_tool\core\branches.py
  - file: git_operations_tool\core\operations.py
  - file: git_operations_tool\core\pull_requests.py
  - file: git_operations_tool\core\repository.py
  - file: git_operations_tool\core